In [ ]:
import json
import boto3

class BedrockClient:
    """
    A client for interacting with the Bedrock Runtime and Bedrock Agent Runtime services.

    Args:
        region_name (str, optional): The AWS region name. Defaults to "us-west-2".
    """

    def __init__(self, region_name="us-west-2"):
        self.runtime_client = boto3.client("bedrock-runtime", region_name=region_name)
        self.agent_runtime_client = boto3.client("bedrock-agent-runtime", region_name=region_name)
        self.anthropic_version = "bedrock-2023-05-31"
        self.model_id = "anthropic.claude-3-haiku-20240307-v1:0"
        self.accept = "application/json"
        self.content_type = "application/json"
        self.knowledge_base_id = "UOJVHFSIEN"
        self.max_tokens = 4096

    def load_prompts(self, system_prompt_file, user_prompt_file):
        """
        Load the system prompt and user prompt from files.

        Args:
            system_prompt_file (str): The path to the file containing the system prompt.
            user_prompt_file (str): The path to the file containing the user prompt.

        Returns:
            tuple: A tuple containing the system prompt and user prompt.
        """
        try:
            with open(system_prompt_file, "r") as file1, open(user_prompt_file, "r") as file2:
                system_prompt = file1.read()
                user_prompt = file2.read()
            return system_prompt, user_prompt
        except FileNotFoundError as e:
            print(f"Error: {e}")
            return None, None
    
    def retrieve_content(self, user_prompt):
        """
        Retrieve relevant content based on the user prompt.

        Args:
            user_prompt (str): The user prompt text.

        Returns:
            str: The retrieved content results in XML format.
        """
        try:
            response = self.agent_runtime_client.retrieve(
                retrievalQuery={'text': user_prompt},
                knowledgeBaseId=self.knowledge_base_id,
                retrievalConfiguration={
                    'vectorSearchConfiguration': {
                        'numberOfResults': 3,
                        # 'overrideSearchType': "HYBRID", #HYBRID OR SEMANTIC
                    }
                }
            )
            results = response['retrievalResults']
            xml_result = self.convert_to_xml(results)
            return xml_result
        except Exception as e:
            print(f"Error: {e}")
            return None

    def convert_to_xml(self, results):
        """
        Convert the retrieved content results to XML format.

        Args:
            results (list): The retrieved content results.

        Returns:
            str: The XML-formatted retrieved content.
        """
        xml_output = "<documents>\n"
        for item in results:
            title = item['metadata']['title']
            text = item['content']['text']
            url = item['metadata']['url']
            
            xml_output += "\t<document>\n"
            xml_output += f"\t\t<title>{title}</title>\n"
            xml_output += f"\t\t<text>{text}</text>\n"
            xml_output += f"\t\t<url>{url}</url>\n"
            xml_output += "\t</document>\n"
        xml_output += "</documents>"
        return xml_output

    def create_request_body(self, system_prompt, retrieved_content, user_message):
        """
        Create the request body for invoking the model.

        Args:
            system_prompt (str): The system prompt text.
            retrieved_content (str): The retrieved content results in XML format.
            user_message (str): The user message text.

        Returns:
            str: The JSON-encoded request body.
        """
        return json.dumps(
            {
                "anthropic_version": self.anthropic_version,
                "max_tokens": self.max_tokens,
                "system": f"""
{system_prompt}

{retrieved_content}
""",
                "messages": [{"role": "user", "content": user_message}],
            }
        )

    def invoke_model(self, request_body):
        """
        Invoke the model with the provided request body.

        Args:
            request_body (str): The JSON-encoded request body.

        Returns:
            str: The content text from the model response.
        """
        try:
            response = self.runtime_client.invoke_model(
                body=request_body,
                modelId=self.model_id,
                accept=self.accept,
                contentType=self.content_type,
            )
            response_body = json.loads(response.get("body").read())
            content_text = response_body["content"][0]["text"]
            return content_text
        except Exception as e:
            print(f"Error: {e}")
            return None


def main():
    """
    The main function to run the Bedrock client example.
    """
    bedrock_client = BedrockClient()
    system_prompt, user_prompt = bedrock_client.load_prompts("system_prompt.txt", "user_prompt.txt")
    
    if system_prompt is None or user_prompt is None:
        print("Failed to load prompts. Exiting.")
        return
    
    retrieved_content = bedrock_client.retrieve_content(user_prompt)
    
    if retrieved_content is None:
        print("Failed to retrieve content. Exiting.")
        return
    
    request_body = bedrock_client.create_request_body(system_prompt, retrieved_content, user_prompt)

    response_text = bedrock_client.invoke_model(request_body)
    
    if response_text is not None:
        print(response_text)


if __name__ == "__main__":
    main()
